In [2]:
import pandas as pd
import yfinance as yf
import os
import mysql.connector

In [3]:
#This function will download and save stock prices of the input tickers as CSV files on the local drive
def download_and_save_ticker_price(tickers, start_date = "2005-01-01"):
    #Check if the ticker already existed in the data folder
    folder_dir = "data/price"
    file_list = os.listdir(folder_dir)
    #Remove the .csv at the end of the file name
    existing_ticker_list = [i[:-4] for i in file_list]

    if type(tickers) == str:
        tickers = [tickers]

    #short list the new tickers
    for i in tickers:
        if i not in existing_ticker_list:
            price = yf.download(i, start = start_date)
            if len(price) > 0:
                price.reset_index(inplace=True)
                price.to_csv('data/price/' + i + '.csv')
                print(i + " is added")
            else:
                print(i + " not found")
        else:
            print(i + " already existed")
     

In [4]:
#This function will download and save stock prices of the input tickers into MySQL local server
def download_ticker_price_and_load_to_mysql(tickers, start_date = "2005-01-01"):
    #Check if the ticker already existed in the data folder
    cnx = mysql.connector.connect(user='root', password='thh2391994',
                              host='127.0.0.1',
                              database='yahoo_finance')
    cursor = cnx.cursor()
    sql_show_table = "SHOW TABLES"
    cursor.execute(sql_show_table)
    table_lst = []
    for i in cursor:
        table_lst.append(i[0])
    
    #UPPERCASE all tickers in the list
    existing_ticker_list = [i.upper() for i in table_lst]

    #If there is only one input ticker as string, convert it into list
    if type(tickers) == str:
        tickers = [tickers]
        
    #UPPERCASE all input tickers in the list
    tickers = [i.upper() for i in tickers]

    #short list the new tickers
    for i in tickers:
        if i not in existing_ticker_list:
            price = yf.download(i, start = start_date)
            if len(price) > 0:
                price.reset_index(inplace=True)

                #Create SQl table for the current ticker
                sql_create_table = (
                    "CREATE TABLE " + i.lower() + " ("
                    "  `date` date NOT NULL,"
                    "  `open` float(2) NOT NULL,"
                    "  `high` float(2) NOT NULL,"
                    "  `low` float(2) NOT NULL,"
                    "  `close` float(2) NOT NULL,"
                    "  `adj_close` float(2) NOT NULL,"
                    "  `volume` int(15) NOT NULL"
                    ") ENGINE=InnoDB")
                cursor.execute(sql_create_table)
                cnx.commit()

                #Insert data to the created table for the current ticker
                val = []
                price_length = len(price)
                for k in range(price_length):
                    lst = price.iloc[k].tolist()
                    lst[-1] = int(lst[-1])
                    tup = tuple(lst)
                    val.append(tup)

                sql_insert_values = "INSERT INTO " + i.lower() + " (date, open, high, low, close, adj_close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"

                cursor.executemany(sql_insert_values, val)
                
                print(i + " is added")
            else:
                print(i + " not found")
        else:
            print(i + " already existed")
            
    cnx.commit()
    cursor.close()
    cnx.close()
     

In [5]:
#Create SQl table
def create_sql_table_for_ticker_price(ticker):
    cnx = mysql.connector.connect(user='root', password='thh2391994',
                                host='127.0.0.1',
                                database='yahoo_finance')
    cursor = cnx.cursor()
    if type(ticker) == str:
        ticker = [ticker]

    for i in ticker:
        price_table = (
            "CREATE TABLE " + i + " ("
            "  `date` date NOT NULL,"
            "  `open` float(2) NOT NULL,"
            "  `high` float(2) NOT NULL,"
            "  `low` float(2) NOT NULL,"
            "  `close` float(2) NOT NULL,"
            "  `adj_close` float(2) NOT NULL,"
            "  `volume` int(15) NOT NULL"
            ") ENGINE=InnoDB")

        cursor.execute(price_table)

    cursor.close()
    cnx.close()
